In [34]:
import pandas as pd
import re
import numpy as np

from glob import glob

In [85]:
# Extract actual stops
size = actual_df.shape[0]
actual_df = pd.read_csv("actual_stops.csv")
actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]

actual_df["succession"] = actual_df["succession"].astype(str)
actual_df["numero_lig"] = actual_df["numero_lig"].astype(str)
actual_df["variante"] = actual_df["variante"].astype(str)
actual_stops = set(actual_df["stop_id"])

# Extract Line stop sequence
extracted_lines = {}

delimiters = np.array(["_" for _ in range(size)])
stops_info = (actual_df["numero_lig"].to_numpy() + delimiters +  actual_df["succession"].to_numpy() + delimiters + actual_df["variante"].to_numpy() + delimiters + actual_df["stop_id"].to_numpy()).tolist()

prev_succession = -999999
direction = ""
for stops in stops_info[::-1]:
    info = stops.split("_")
    if int(prev_succession) < int(info[1]):
        direction = info[3]
    
    data = extracted_lines.setdefault(info[0], {})
    
    succession_info = data.setdefault(info[3], {})
    succession_info[direction] = info[1]
    
    data[info[3]] = succession_info
    extracted_lines[info[0]] = data
    
    line_direction = data.setdefault("direction", {})
    visited_stops = line_direction.setdefault(direction, set())
    visited_stops.add(info[3])
    
    line_direction[direction] = visited_stops
    data["direction"] = line_direction
    
    prev_succession = info[1]

# Pair stops with the succession line, 
# if there's multiple sucession line check whether direction is the same
# If not, try to find direction id in the direction stops
# otherwise set sucession to 99999

lines = []
for file in glob("./Lines_vehiclePositions/*.csv"):
    if "null" in file or "71" not in file:
        continue
    df = pd.read_csv(file);
    
    df["lineID"] = df["lineID"].astype(str)
    df["directionID"] = df["directionID"].astype(str)
    df["pointID"] = df["pointID"].astype(str)
    
    delimiters = np.array(["_" for _ in range(df.shape[0])])
    stops_infos = (df["lineID"].to_numpy() + delimiters + df["directionID"].to_numpy() + delimiters + df["pointID"].to_numpy() + delimiters + df.index.astype(str).values).tolist()
    
    succession_arr = []
    for stop in stops_infos:
        info = stop.split("_")
        data = extracted_lines[info[0]]
        
        succession = 999999
        if info[2] not in data:
            succession_arr.append(succession)
            continue
        
        succession_info = data[info[2]]
        if info[1] not in succession_info:
            succession_arr.append(succession)
            continue
        
        succession = succession_info[info[1]]
        succession_arr.append(succession)
    
    df['succession'] = np.array(succession_arr)
    
    df["lineID"] = df["lineID"].astype(int)
    df["directionID"] = df["directionID"].astype(int)
    df["pointID"] = df["pointID"].astype(int)
    df["succession"] = df["succession"].astype(int)
    
    df.sort_values(["directionID", "succession", "time"], ascending=[True, True, True]).to_csv("line_71_sucession.csv")
    